In [1]:
# !python.exe -m pip install --upgrade pip
# %pip install -U pip

# !conda clean --all
# !conda update --all

In [23]:
import sys
import torch
import subprocess

print(f"\nDados do ambiente local")
# !nvcc --version
def verificar_versoes():
    """Verifica e exibe as versões  da GPU, do Python e bibliotecas CUDA instaladas no sistema."""

    # Versão do Python
    python_version = sys.version.split()[0]
    print(f"   Versão Python: {python_version}")
    print(f"  Versão PyTorch: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"  Versão do CUDA: {torch.version.cuda}")
        print(f" Versão do cuDNN: {torch.backends.cudnn.version()}")
    else:
        print("CUDA não disponível, não está configurado corretamente.")

    # Versão do CUDA (usando nvcc)
    try:
        cuda_version = subprocess.check_output(["nvcc", "--version"])
        cuda_version_l3 = [' '.join(cuda_version.decode("utf-8").strip().split("\n")[-2].split(',')[1:])][0].strip()
        cuda_version_l4 = cuda_version.decode("utf-8").strip().split("\n")[-1]
        print(f"    CUDA Toolkit: {cuda_version_l3}")
        print(f"      Build nvcc: {cuda_version_l4}")
    except FileNotFoundError:
        print("nvcc não encontrado. CUDA pode não estar instalado ou configurado corretamente.")

# Chamando a função para verificar as versões
verificar_versoes()

print(f"\nDetalhes da biblioteca PyTorch")
!pip3 show torch


Dados do ambiente local
   Versão Python: 3.11.2
  Versão PyTorch: 2.3.1+cu121
  Versão do CUDA: 12.1
 Versão do cuDNN: 8907
    CUDA Toolkit: release 12.5  V12.5.82
      Build nvcc: Build cuda_12.5.r12.5/compiler.34385749_0

Detalhes da biblioteca PyTorch
Name: torch
Version: 2.3.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: c:\Users\marco\.conda\envs\beakerx\Lib\site-packages
Requires: filelock, fsspec, jinja2, mkl, networkx, sympy, typing-extensions
Required-by: fairseq, sentence-transformers, spacy-transformers, torchaudio, torchvision, xformers


# Execução 01: Sem mecanismo de atenção

In [24]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import time

# Carregar modelo e tokenizer pré-treinados
model_name = "bert-base-uncased"  # Escolha o modelo desejado
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to("cuda")  # Mover para GPU

# Textos de entrada (exemplo)
texts = [
    "This is a sentence about natural language processing.",
    "Another sentence about NLP.",
    "An unrelated sentence about dogs."
]

# Pré-processamento
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to("cuda")

# Medir tempo de execução
start_time = time.time()

# Embedding
with torch.no_grad():
    embeddings = model(**encoded_input)["pooler_output"].cpu().numpy()

# Cálculo de similaridade
similarity_matrix = cosine_similarity(embeddings)

end_time = time.time()

print("Matriz de similaridade:")
print(similarity_matrix)

print(f"Tempo de execução: {end_time - start_time:.2f} segundos")

Matriz de similaridade:
[[0.99999976 0.9558643  0.9794879 ]
 [0.9558643  1.         0.9838272 ]
 [0.9794879  0.9838272  0.9999999 ]]
Tempo de execução: 1.25 segundos


c:\Users\marco\.conda\envs\beakerx\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


# Execução 02: Com mecanismo de atenção

A versão do CUDA precisa ser compatível com a Flash Attention. A Flash Attention foi introduzida no PyTorch 2.0 e requer CUDA 11.8 ou superior.

In [25]:
!conda uninstall pytorch torchvision torchaudio

Solving environment: ...working... failed



PackagesNotFoundError: The following packages are missing from the target environment:
  - pytorch
  - torchaudio
  - torchvision




Executar no Terminal:

    conda create -n pytorch_env python=3.11
    conda activate pytorch_env

In [ ]:
# Sempre atualizar versões mais recentes em https://pytorch.org/
%conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch-nightly -c nvidia

In [ ]:
# Chamando a função para verificar as versões
verificar_versoes()

print(f"\nDetalhes da biblioteca PyTorch")
!pip3 show torch

In [ ]:
# !conda config --add channels conda-forge
# !conda config --add channels rapidsai
# !conda config --add channels nvidia

# %conda install -c rapidsai -c nvidia -c conda-forge pylibcugraph=24.6.1 cudatoolkit=12.5
%conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch-nightly -c nvidia

In [ ]:
import pandas as pd
import networkx as nx

url = "https://data.rapids.ai/cugraph/datasets/cit-Patents.csv"
df = pd.read_csv(url, sep=" ", names=["src", "dst"], dtype="int32")
G = nx.from_pandas_edgelist(df, source="src", target="dst")

%time result = nx.betweenness_centrality(G, k=10)

In [ ]:
%conda install -c rapidsai -c nvidia pylibcugraph

In [ ]:
%pip3 install --extra-index-url https://pypi.nvidia.com/pylibcugraph-cu12 pylibcugraph_cu12-24.6.1-cp311-cp311-manylinux_2_28_x86_64.whl